<a href="https://colab.research.google.com/github/dawidstajszczyk/Recommender-system/blob/main/recommender_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Wczytanie danych

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Wczytaj dostępne filmy
movies = pd.read_csv('https://s3-us-west-2.amazonaws.com/recommender-tutorial/movies.csv')
movies.head()



,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [2]:
# Wczytaj dostępne oceny
ratings = pd.read_csv('https://s3-us-west-2.amazonaws.com/recommender-tutorial/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


Implementacja *user-item matrix*

In [3]:
from scipy.sparse import csr_matrix

def user_item_matrix(df):

  # Pobierz wymiary macierzy
  rows_num = df['userId'].nunique()
  columns_num = df['movieId'].nunique()

  # Pobierz unikalne ID użytkowników i filmów
  unique_users = np.unique(df["userId"])
  unique_movies = np.unique(df["movieId"])

  # Utwórz mapper dla użytkowników (which userId correspond to which row 'utility' matrix)
  user_mapper = {user_id: index for index, user_id in enumerate(unique_users)}

  # Utwórz mapper dla filmów (which movieId correspond to which column 'utility' matrix)
  movie_mapper = {movie_id: index for index, movie_id in enumerate(unique_movies)}

  # Utwórz mapper odwrotny dla użytkowników
  user_inv_mapper = {index: user_id for index, user_id in enumerate(unique_users)}

  # Utwórz mapper odwrotny dla filmów
  movie_inv_mapper = {index: movie_id for index, movie_id in enumerate(unique_movies)}

  # Pobierz indeksy użytkowników i filmów
  user_indices = [user_mapper[i] for i in df['userId']]
  item_indices = [movie_mapper[i] for i in df['movieId']]

  # Utwórz user-item matrix
  X = csr_matrix((df["rating"], (user_indices, item_indices)), shape=(rows_num, columns_num))

  return X, user_mapper, movie_mapper, user_inv_mapper, movie_inv_mapper



Podgląd *user-item matrix*

In [4]:
# Utwórz user-item matrix (X)
X, user_mapper, movie_mapper, user_inv_mapper, movie_inv_mapper = user_item_matrix(ratings)

# Pobierz fragment macierzy X
data = X[:5, :5].toarray()

# Utwórz DataFrame na podstawie danych 'data'
df = pd.DataFrame(data)
df.index.name = 'user'
df.columns.name = 'item'

# Wyświetl fragment macierzy z opisanymi osiami
print(df)

item    0    1    2    3    4
user                         
0     4.0  0.0  4.0  0.0  0.0
1     0.0  0.0  0.0  0.0  0.0
2     0.0  0.0  0.0  0.0  0.0
3     0.0  0.0  0.0  0.0  0.0
4     4.0  0.0  0.0  0.0  0.0


**Collaborative Filtering**

In [5]:
from sklearn.neighbors import NearestNeighbors

def find_similar_movies(movie_id, movie_mapper, movie_inv_mapper, X, k, metric = 'cosine'):
  # Pobierz indeks wybranego filmu
  movie_index = movie_mapper[movie_id]

  # Pobierz wektor cech (ocen) dla wybranego filmu
  X = X.T
  movie_vector = X[movie_index]

  # Jeśli movie_vector jest tablicą numpy, spłasz ją do jednego wymiaru
  if isinstance(movie_vector, (np.ndarray)):
    movie_vector = movie_vector.reshape(1,-1)


  # Zainicjuj obiekt NearestNeighbors
  kNN = NearestNeighbors(n_neighbors= k + 1, algorithm="brute", metric=metric)

  # Dopasuj model k-Nearest-Neighbours do danych
  kNN.fit(X)

  # Znajdź k najbliższych sąsiadów dla wybranego filmu
  neighbour = kNN.kneighbors(movie_vector, return_distance=False)

  # Zainicjuj listę do przechowywania indeksów najbliższych sąsiadów
  neighbour_indices = []

  # Pobierz movieId wybrane przez algorytm kNN
  for i in range(0,k):
    n = neighbour.item(i)
    neighbour_indices.append(movie_inv_mapper[n])

  # Usuń film, dla którego przeprowadzana jest rekomendacja
  neighbour_indices.pop(0)

  return neighbour_indices



Dokonanie rekomendacji

In [6]:
similar_movies = find_similar_movies(1, movie_mapper, movie_inv_mapper, X, k=10)
similar_movies

[3114, 480, 780, 260, 356, 364, 1210, 648, 1265]

In [7]:
# Pobierz listę identyfikatorów wszystkich filmów i tytułów
movie_ids = movies['movieId']
titles = movies['title']

# Zainicjuj listę do przechowywania wybranych tytułów
movie_titles = {}

# Dodaj pary movieId-title do słownika
for movie_id, title in zip(movie_ids, titles):
    movie_titles[movie_id] = title

# Znajdź filmy podobne do filmu o identyfikatorze movie_id
movie_id = 100
similar_movies = find_similar_movies(movie_id, movie_mapper, movie_inv_mapper, X, k=10)


print(f"Na podstawie filmu {movie_titles[movie_id]}:")
for movie_id in similar_movies:
    print(movie_titles[movie_id])

Na podstawie filmu City Hall (1996):
Othello (1995)
Cobb (1994)
Chamber, The (1996)
Eye for an Eye (1996)
Bloodsport 2 (a.k.a. Bloodsport II: The Next Kumite) (1996)
Bed of Roses (1996)
Dangerous Ground (1997)
Candidate, The (1972)
First Kid (1996)


In [8]:
movies["genres"].head()

0    Adventure|Animation|Children|Comedy|Fantasy
1                     Adventure|Children|Fantasy
2                                 Comedy|Romance
3                           Comedy|Drama|Romance
4                                         Comedy
Name: genres, dtype: object

**Content-based filtering**

Utworzenie macierzy *movie_genres*

In [9]:
# Utwórz zestaw wszystkich gatunków filmowych
genres = set()
for genre_list in movies['genres']:
    for genre in genre_list.split('|'):
        genres.add(genre)

# Dodaj nową kolumnę z gatunkiem 'genre' do ramki danych 'movies'.
# Wartość w tej kolumnie to 1, jeśli film należy do danego gatunku, a 0 w przeciwnym przypadku.
for genre in genres:
    movies[genre] = movies['genres'].transform(lambda x: int(genre in x))

# Utwórz kopię DataFrame'u i usuń wymienione kolumny
movie_genres = movies.drop(columns=['movieId', 'title', 'genres'])

# Podejrzyj fragment DataFrame'u
movie_genres.head()


,(no genres listed),Mystery,Fantasy,Adventure,Film-Noir,Action,Documentary,Romance,Crime,Comedy,Thriller,Drama,War,Children,Animation,Musical,IMAX,Sci-Fi,Western,Horror
0,0,0,1,1,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0
1,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


Utworzenie *Cosine Similarity Matrix*

In [10]:
from sklearn.metrics.pairwise import cosine_similarity

# Oblicz cosine_similatiry
# cosine_sim[i, j] będzie reprezentować podobieństwo kosinusowe między i-tym a j-tym filmem.
cosine_sim = cosine_similarity(movie_genres, movie_genres)

print(f"Dimensions of our genres cosine similarity matrix: {cosine_sim.shape}")

Dimensions of our genres cosine similarity matrix: (9742, 9742)


In [11]:
# Wybierz tytuł filmu.
title = 'Waiting to Exhale (1995)'

# Utwórz słownik. Klucz - tytuł filmu, wartość - indeks filmu
movie_idx = dict(zip(movies['title'], list(movies.index)))

# Przypisz indeks wybranego filmu
idx = movie_idx[title]

print(f"Movie index for '{title}' is: {idx}")

Movie index for 'Waiting to Exhale (1995)' is: 3


In [18]:
# Ustal liczbę rekomendacji
n_recommendations=10

# Wyodrębnij podobieństwo kosinusowe pomiędzy wybranym filmem, a pozostałymi
sim_scores = list(enumerate(cosine_sim[idx]))
print("Part of the list 'sim_scores': ",sim_scores[:5])

# Sortuj malejąco listę 'sim_scores'
sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
sim_scores = sim_scores[1:(n_recommendations+1)]
similar_movies = [i[0] for i in sim_scores]

Part of the list 'sim_scores':  [(0, 0.25819888974716115), (1, 0.0), (2, 0.816496580927726), (3, 1.0000000000000002), (4, 0.5773502691896258)]


In [13]:
print(f"Because you watched {title}:")
movies['title'].iloc[similar_movies]

Because you watched Waiting to Exhale (1995):


10                  American President, The (1995)
47                         Mighty Aphrodite (1995)
52               Postman, The (Postino, Il) (1994)
83                          Beautiful Girls (1996)
165                 Something to Talk About (1995)
191                        Don Juan DeMarco (1995)
198    Eat Drink Man Woman (Yin shi nan nu) (1994)
243                           Nobody's Fool (1994)
309                        Corrina, Corrina (1994)
317                     I Like It Like That (1994)
Name: title, dtype: object